![](https://img.kaikeba.com/web/kkb_index/img_index_logo.png)

# Lesson-02 Assignment

   本周课程主要包含了图像基本操作中的卷积/滤波，以及最基础的特征描述算法————Harris Corner和SIFT。希望大家认真复习，理解图像求导/卷积的目的，以及特征的本质。
   本次作业分为阅读部分和算法部分。

## 本次作业的内容

### [Reading]:

You needn't finish reading all of them in just one week!
It's just good for you to know what's happening in this area and to figure out how people try to improve SIFT.

You needn't to remember all of them. 
But please DO REMEMBER procedures of SIFT and HoG. For those who're interested in SLAM, Orb is your inevitable destiny.

1. [optional] Bilateral Filter: https://blog.csdn.net/piaoxuezhong/article/details/78302920
2. Feature Descriptors:
   [Compulsory]
   Hog: https://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf
   SURF: https://www.vision.ee.ethz.ch/~surf/eccv06.pdf
   [optional]
   BRISK: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.371.1343&rep=rep1&type=pdf
   Orb: http://www.willowgarage.com/sites/default/files/orb_final.pdf [Compulsory for SLAM Guys]
3. Preview parts:
   K-Means: I have no doubts about what you are going to read and where you gonna find the reading materials. There are tons of papers/blogs describing k-means. Just grab one and read.We'll talk about this topic in 3 weeks.

### [Coding]:
Finish 2D convolution/filtering by your self. 
What you are supposed to do can be described as "median blur", which means by using a sliding window on an image, your task is not going to do a normal convolution, but to find the median value within that crop.

You can assume your input has only one channel. (a.k.a a normal 2D list/vector) And you do need to consider the padding method and size. There are 2 padding ways: REPLICA & ZERO. When "REPLICA" is given to you, the padded pixels are same with the border pixels. E.g is [1 2 3] is your image, the padded version will be  [[(...1 1) 1 2 3 (3 3...)]  where how many 1 & 3 in the parenthesis depends on your padding size. When "ZERO", the padded version will be [(...0 0) 1 2 3 (0 0...)]

Assume your input's size of the image is W x H, kernel size's m x n. You may first complete a version with O(W·H·m·n log(m·n)) to O(W·H·m·n·m·n)).

Follow up 1: Can it be completed in a shorter time complexity?

In [2]:
def mid_trans(channel, kernel_size, fill=0):
    total_length = kernel_size * kernel_size
    offset = kernel_size // 2
    source = channel.copy()
    y_length, x_length = source.shape
    for y_index in range(y_length):
        for x_index in range(x_length):
            x_start = x_index - 1
            y_start = y_index - 1
            x_start = x_index if x_start < 0 else x_start
            y_start = y_index if y_start < 0 else y_start
            x_end = min(x_length, x_index + offset + 1)
            y_end = min(y_length, y_index + offset + 1)
            data = channel[y_start:y_end, x_start:x_end].ravel()
            add_zero = total_length - len(data)
            # 补0
            for _ in range(add_zero):
                data = np.append(data, fill)
            mid_value = np.median(data)
            source[y_index,x_index] = mid_value
    return source

def mediaBlur(image, to_rgb=None, kernel_size=3, fill=0):
    if to_rgb is not None:
        image = cv2.cvtColor(image, to_rgb)
    
    rc, gc, bc = cv2.split(image)
    mid_r = mid_trans(rc, kernel_size, fill)
    mid_g = mid_trans(gc, kernel_size, fill)
    mid_b = mid_trans(bc, kernel_size, fill)
    return cv2.merge((mid_r, mid_g, mid_b))

### [Reading + Pseudo Code]:
We haven't told RANSAC algorithm this week. So please try to do the reading.

And now, we can describe it here:
    We have 2 sets of points, say, Points A and Points B. We use A.1 to denote the first point in A, B.2 the 2nd point in B and so forth. Ideally, A.1 is corresponding to B.1, ... A.m corresponding B.m. However, it's obvious that the matching cannot be so perfect and the matching in our real world is like: 
    A.1-B.13, A.2-B.24, A.3-x (has no matching), x-B.5, A.4-B.24(This is a wrong matching) ...
    The target of RANSAC is to find out the true matching within this messy.
    
Algorithm for this procedure can be described like this:
    1. Choose 4 pair of points randomly in our matching points. Those four called "inlier" (中文： 内点) while others "outlier" (中文： 外点)
    2. Get the homography of the inliers
    3. Use this computed homography to test all the other outliers. And separated them by using a threshold into two parts:
        a. new inliers which is satisfied our computed homography
        b. new outliers which is not satisfied by our computed homography.
    4. Get our all inliers (new inliers + old inliers) and goto step 2
    5. As long as there's no changes or we have already repeated step 2-4 k, a number actually can be computed, times, we jump out of the recursion. The final homography matrix will be the one that we want.

[WARNING!!! RANSAC is a general method. Here we add our matching background to that.]

Your task: please complete pseudo code (it would be great if you hand in real code!) of this procedure.

Python:
def ransacMatching(A, B):
    A & B: List of List

Follow up 1. For step 3. How to do the "test“? Please clarify this in your code/pseudo code
Follow up 2. How do decide the "k" mentioned in step 5. Think about it mathematically!

You are supposed to hand in the code in 1 week.

# RANSAC伪代码：
## 1. 选取部分特征点，并初始化模型
$$
\Large f(X_0) \Rightarrow Y_0
$$

## 2. 根据误差，对各点进行分类
- 误差: $\Large \epsilon$
$$
\left\{
\begin{matrix}
f(x_i) - Y_i > \epsilon & false \\
f(x_i) - Y_i \le \epsilon & true
\end{matrix}
\right.
$$

记录命中的值，也就是$true$的点的分类。不要忘记第一步中选取的初始化点哦。

## 3. 判断和返回
- 迭代次数：$N$

步骤如下

- 如果不满足迭代次数，更新最多命中模型(点集)，然后到第一步重复执行
- 达到迭代次数，返回最多命中模型(点集合)，如果都没有，返回none


```python
def mapping(X, Y, times, threshold, want):
    result = []
    _times = 0
    while len(result) < want or _times < times:
        sample_x, sample_y = choose_sample(X, Y)
        sample_func = get_sanple_function(sample_x, sample_y)
        result = np.append(result, (sample_func, len(sample_func(X)[sample_func(X) - Y < threshold])))
    return None if not result else np.sort(result)[-1]
                           
```

### [作业截止时间]

作业能帮助你回顾课堂内容，你又可以通过作业进行代码实操。咱们可要认真、及时的完成作业哦！自布置作业起两周内提交，助教及时批改作业哦～逾期提交不批改。（特殊情况，请找班主任请假。）

这次的作业就到这里了！祝大家学习进步！

![image alt <](http://5b0988e595225.cdn.sohucs.com/images/20190420/1d1070881fd540db817b2a3bdd967f37.gif)